## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-15-08-43-19 +0000,bbc,Brazil's Lula hits back at Trump over Bolsonar...,https://www.bbc.com/news/articles/cj3yxkkdlkvo...
1,2025-09-15-08-38-50 +0000,nyt,Tariffs Are Uniting Two Whiskey Heavyweights: ...,https://www.nytimes.com/2025/09/15/business/tr...
2,2025-09-15-08-36-24 +0000,nyt,Did Britain Win the Trade War? Why It’s Tough ...,https://www.nytimes.com/2025/09/15/business/ec...
3,2025-09-15-08-35-34 +0000,bbc,Rubio meets Netanyahu to discuss fallout from ...,https://www.bbc.com/news/articles/czxw4xkxg51o...
4,2025-09-15-08-25-48 +0000,bbc,Starmer facing leadership questions after Mand...,https://www.bbc.com/news/articles/ce32qepq12qo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2317/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
3,trump,25
258,kirk,22
257,charlie,17
141,suspect,9
86,will,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
134,2025-09-14-15-50-46 +0000,nypost,Trump still furious over Charlie Kirk’s murder...,https://nypost.com/2025/09/14/us-news/utah-gov...,90
152,2025-09-14-13-57-20 +0000,nypost,Charlie Kirk suspect Tyler Robinson not cooper...,https://nypost.com/2025/09/14/us-news/charlie-...,80
65,2025-09-14-23-51-33 +0000,nypost,Trump blasts ‘the left’ and announces ‘major i...,https://nypost.com/2025/09/14/us-news/trump-bl...,78
107,2025-09-14-19-31-04 +0000,nypost,Eric Trump: Assassination of Charlie Kirk has ...,https://nypost.com/2025/09/14/us-news/eric-tru...,76
76,2025-09-14-22-55-24 +0000,nypost,"2 men arrested for allegedly planting, lightin...",https://nypost.com/2025/09/14/us-news/2-men-ar...,70


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
134,90,2025-09-14-15-50-46 +0000,nypost,Trump still furious over Charlie Kirk’s murder...,https://nypost.com/2025/09/14/us-news/utah-gov...
184,38,2025-09-14-09-30-00 +0000,wsj,In the weeks since President Trump’s 50% tarif...,https://www.wsj.com/economy/trade/global-trade...
158,33,2025-09-14-13-00-00 +0000,wsj,"As America’s 250th birthday approaches, Robert...",https://www.wsj.com/business/robert-reed-churc...
3,31,2025-09-15-08-35-34 +0000,bbc,Rubio meets Netanyahu to discuss fallout from ...,https://www.bbc.com/news/articles/czxw4xkxg51o...
7,31,2025-09-15-08-08-07 +0000,wsj,Stock Market Today: Dow Futures Inch Up; U.S.-...,https://www.wsj.com/livecoverage/stock-market-...
26,30,2025-09-15-04-43-10 +0000,nypost,Maniac NYC driver who mowed down teen girl aft...,https://nypost.com/2025/09/15/us-news/maniac-n...
47,29,2025-09-15-02-00-00 +0000,wsj,A bullet casing found after the killing of Cha...,https://www.wsj.com/politics/policy/fascist-en...
20,27,2025-09-15-06-35-58 +0000,nypost,UK police say 49-year-old German remains a sus...,https://nypost.com/2025/09/15/world-news/uk-po...
96,24,2025-09-14-20-20-26 +0000,nypost,Trump reveals his prized White House ballroom ...,https://nypost.com/2025/09/14/us-news/presiden...
77,23,2025-09-14-22-49-19 +0000,nypost,Roaming pack of wild dogs — as many as 40 — te...,https://nypost.com/2025/09/14/us-news/roaming-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
